# SQLite Tutorial: Ground Motion Database

## What is SQLite?

**SQLite** is a lightweight, serverless, self-contained SQL database engine that stores data in a single file. It's perfect for:
- **Small to medium applications** (websites, mobile apps, desktop software)
- **Prototyping and development** (no server setup required)
- **Data analysis and research** (portable databases)
- **Embedded systems** (IoT devices, mobile applications)

### Key Advantages of SQLite:
- ✅ **Zero Configuration**: No server installation or management needed
- ✅ **Cross-Platform**: Runs on Windows, macOS, Linux, Android, iOS
- ✅ **ACID Compliant**: Atomic, Consistent, Isolated, Durable transactions
- ✅ **Small Footprint**: Library is less than 1MB
- ✅ **Fast**: Optimized for read operations and small datasets
- ✅ **Reliable**: Extensively tested, used by millions of applications

### When to Use SQLite vs Other Databases:

| Feature | SQLite | PostgreSQL | MySQL |
|---------|--------|------------|-------|
| **Setup** | No setup required | Server installation | Server installation |
| **Concurrent Writes** | Limited | Excellent | Good |
| **Max Database Size** | 281 TB | Unlimited | Depends on storage |
| **Use Case** | Development, small apps | Large applications | Web applications |
| **Hosting** | File-based | Server required | Server required |

---

## Database Design Fundamentals

This tutorial demonstrates a **relational database** for earthquake ground motion data with three interconnected tables:

1. **`event`** - Earthquake event information
2. **`station`** - Recording station details  
3. **`motion`** - Ground motion recordings (links events and stations)

### Database Schema Overview:
```
event (1) ←→ (many) motion (many) ←→ (1) station
```

This is a **many-to-many relationship** through the `motion` table:
- One earthquake can be recorded at multiple stations
- One station can record multiple earthquakes
- Each recording has unique motion characteristics

## Step 1: Connecting to SQLite Database

### Key Concepts:

**`sqlite3.connect()`** creates or opens a database file:
- If the file doesn't exist, SQLite creates it automatically
- Returns a **Connection object** for database operations
- **Best Practice**: Use context managers (`with` statement) for automatic cleanup

**`connection.cursor()`** creates a cursor object:
- **Cursor** = pointer that executes SQL commands and fetches results
- Think of it as your "command interface" to the database
- Multiple cursors can exist per connection

### Error Handling in Database Operations:

The `exec()` function below demonstrates proper error handling:
- **`try/except`** blocks catch SQL errors (syntax, constraint violations, etc.)
- **`sqlite3.Error`** is the base exception class for all SQLite errors
- Always handle errors gracefully in production code

In [2]:
import sqlite3
from sqlite3 import Error

# Create a new database file called 'gmdatabase.db' and create a cursor
con = sqlite3.connect('gmdatabase.db')
cur = con.cursor()

# Define a function that runs a query and returns an error if it fails
def exec(cur, query):
    try:
        cur.execute(query)
    except Error as e:
        print(e)

## Step 2: Creating Tables with SQL DDL

### Understanding CREATE TABLE Syntax:

**DDL (Data Definition Language)** defines database structure:
```sql
CREATE TABLE IF NOT EXISTS table_name (
    column_name data_type constraints,
    column_name data_type constraints,
    ...
);
```

### SQLite Data Types:
- **`INTEGER`** - Whole numbers (1, 42, -17)
- **`REAL`** - Floating point numbers (3.14, -0.5, 1.2e5)
- **`TEXT`** - Strings ("earthquake", 'station name')
- **`BLOB`** - Binary data (images, files)
- **`NULL`** - Missing/unknown values

### Key Constraints Explained:

1. **`PRIMARY KEY`** - Unique identifier for each row
   - Automatically creates an index for fast lookups
   - Cannot be NULL or duplicate
   - Use `INTEGER PRIMARY KEY` for auto-incrementing IDs

2. **`FOREIGN KEY`** - Links tables together
   - References the primary key of another table
   - Maintains **referential integrity** (prevents orphaned records)
   - Example: `motion.event_id` must exist in `event.event_id`

3. **`IF NOT EXISTS`** - Safe table creation
   - Only creates table if it doesn't already exist
   - Prevents errors when running scripts multiple times

### Database Schema Analysis:

Our three tables demonstrate a classic **many-to-many relationship**:
- **`event`**: Stores earthquake information (magnitude, location)
- **`station`**: Stores recording station details (location, soil properties)  
- **`motion`**: Junction table linking events to stations with measurement data

In [3]:
# Create tables. Note that the motion table has two foreign keys: event_id and station_id

query = 'CREATE TABLE IF NOT EXISTS event ( '
query += 'event_id integer primary key, '
query += 'event_name text, '
query += 'magnitude real, '
query += 'epicentral_latitude real, '
query += 'epicentral_longitude real)'
exec(cur, query)
    
query = 'CREATE TABLE IF NOT EXISTS station ( '
query += 'station_id integer primary key, '
query += 'station_name text, '
query += 'station_latitude real, '
query += 'station_longitude real, '
query += 'vs30 real)'
exec(cur, query)
    
query = 'CREATE TABLE IF NOT EXISTS motion ( '
query += 'motion_id integer primary key, '
query += 'event_id INTEGER, '
query += 'station_id INTEGER, '
query += 'rjb real, '
query += 'pga real, '
query += 'FOREIGN KEY(event_id) REFERENCES event(event_id), '
query += 'FOREIGN KEY(station_id) REFERENCES station(station_id))'
exec(cur, query)

## Step 3: Inserting Data with SQL DML

### Understanding INSERT Syntax:

**DML (Data Manipulation Language)** modifies database content:
```sql
INSERT INTO table_name (column1, column2, ...) 
VALUES (value1, value2, ...);
```

### Best Practices for Data Insertion:

1. **Explicit Column Names**: Always specify column names for clarity
   - `INSERT INTO event (event_id, event_name, ...)` ✅
   - `INSERT INTO event VALUES (1, "name", ...)` ❌ (fragile)

2. **Data Type Matching**: Ensure values match column types
   - `magnitude real` expects numbers: `6.3` ✅, `"6.3"` works but not ideal
   - `event_name text` expects strings: `"Westwood Hills"` ✅

3. **Primary Key Management**: 
   - We're manually setting IDs here for simplicity
   - In production, often use `INTEGER PRIMARY KEY` for auto-increment
   - SQLite automatically handles ID assignment if you omit the ID column

### Data Validation Tips:
- **Coordinates**: Latitude (-90 to 90), Longitude (-180 to 180)
- **Magnitude**: Typically 0-10 scale (Richter/Moment magnitude)
- **PGA (Peak Ground Acceleration)**: Usually 0-2g for most earthquakes
- **Vs30**: Soil shear wave velocity, typically 150-1500 m/s

In [4]:
# Insert data into event table

query = 'INSERT INTO event (event_id, event_name, magnitude, epicentral_latitude, epicentral_longitude) VALUES (1, "Westwood Hills", 6.3, 34.0689, -118.4452)'
exec(cur, query)
query = 'INSERT INTO event (event_id, event_name, magnitude, epicentral_latitude, epicentral_longitude) VALUES (2, "Hollywood Valley", 7.2, 34.1027, -118.3404)'
exec(cur, query)

UNIQUE constraint failed: event.event_id


## Step 4: Querying Data with SELECT

### Understanding SELECT Syntax:

```sql
SELECT column1, column2, ... 
FROM table_name 
WHERE condition;
```

**`SELECT *`** retrieves all columns from a table:
- Useful for exploration and small datasets
- In production, specify exact columns for better performance
- Example: `SELECT event_name, magnitude FROM event`

**`cur.fetchall()`** retrieves all query results:
- Returns a list of tuples, one per row
- Alternative methods: `fetchone()` (single row), `fetchmany(n)` (n rows)
- Always call fetch methods after executing SELECT queries

### Query Optimization Tips:
- **Use indexes**: Primary keys are automatically indexed
- **Limit results**: Add `LIMIT 100` for large datasets
- **Filter early**: Use `WHERE` clauses to reduce data transfer
- **Select specific columns**: Avoid `SELECT *` in production code

In [5]:
# Query event table to make sure the data are entered correctly

query = 'SELECT * FROM event'
exec(cur, query)
cur.fetchall()

[(1, 'San Andreas Test', 5.8, 34.0522, -118.2437),
 (2, 'Hollywood Valley', 7.2, 34.1027, -118.3404)]

In [6]:
# Insert data into station table

query = 'INSERT INTO station (station_id, station_name, station_latitude, station_longitude, vs30) VALUES (1, "Factor Building", 34.006677, -118.442007, 450)'
exec(cur, query)
query = 'INSERT INTO station (station_id, station_name, station_latitude, station_longitude, vs30) VALUES (2, "Santa Monica Courthouse", 34.0108, -118.4903, 275)'
exec(cur, query)

## Step 5: Inserting Station Data

### Understanding Geospatial Data:

**Station Location Data**:
- **Latitude/Longitude**: Geographic coordinates in decimal degrees
- **Vs30**: Average shear wave velocity in top 30 meters of soil
  - Important for ground motion prediction
  - Higher Vs30 = stiffer soil = typically lower amplification

**Real-World Context**:
- Factor Building: Research facility near UCLA
- Santa Monica Courthouse: Government building in Santa Monica
- Both locations represent different soil conditions (Vs30 values)

In [7]:
# Insert data into motion table

query = 'INSERT INTO motion (motion_id, event_id, station_id, rjb, pga) VALUES (1, 1, 1, 2.0, 0.84)'
exec(cur, query)
query = 'INSERT INTO motion (motion_id, event_id, station_id, rjb, pga) VALUES (2, 1, 2, 14.0, 0.28)'
exec(cur, query)
query = 'INSERT INTO motion (motion_id, event_id, station_id, rjb, pga) VALUES (3, 2, 1, 20.0, 0.61)'
exec(cur, query)
query = 'INSERT INTO motion (motion_id, event_id, station_id, rjb, pga) VALUES (4, 2, 2, 30.0, 0.32)'
exec(cur, query)

## Step 6: Recording Ground Motion Data

### Understanding Ground Motion Parameters:

**RJB (Joyner-Boore Distance)**:
- Distance from recording site to closest point on fault rupture surface
- Critical parameter for ground motion prediction equations
- Measured in kilometers

**PGA (Peak Ground Acceleration)**:
- Maximum acceleration recorded during earthquake
- Expressed as fraction of gravity (g = 9.81 m/s²)
- Key parameter for structural engineering and seismic hazard

### Data Relationships:
Each motion record connects:
1. **One earthquake event** (via event_id foreign key)
2. **One recording station** (via station_id foreign key)
3. **Unique ground motion measurements** for that event-station pair

This creates a complete record of "what was recorded where during which earthquake"

In [8]:
# Run a query that joins the tables on primary / foreign key constraints

query = 'SELECT * FROM motion JOIN station ON motion.station_id = station.station_id JOIN event ON motion.event_id = event.event_id'
exec(cur, query)
cur.fetchall()

[(1,
  1,
  1,
  2.0,
  0.84,
  1,
  'Factor Building',
  34.006677,
  -118.442007,
  450.0,
  1,
  'San Andreas Test',
  5.8,
  34.0522,
  -118.2437),
 (2,
  1,
  2,
  14.0,
  0.28,
  2,
  'Santa Monica Courthouse',
  34.0108,
  -118.4903,
  275.0,
  1,
  'San Andreas Test',
  5.8,
  34.0522,
  -118.2437),
 (3,
  2,
  1,
  20.0,
  0.61,
  1,
  'Factor Building',
  34.006677,
  -118.442007,
  450.0,
  2,
  'Hollywood Valley',
  7.2,
  34.1027,
  -118.3404),
 (4,
  2,
  2,
  30.0,
  0.32,
  2,
  'Santa Monica Courthouse',
  34.0108,
  -118.4903,
  275.0,
  2,
  'Hollywood Valley',
  7.2,
  34.1027,
  -118.3404)]

## Step 7: Advanced Querying with JOINs

### Understanding JOIN Operations:

**JOIN** combines data from multiple tables based on related columns:
```sql
SELECT columns
FROM table1 
JOIN table2 ON table1.key = table2.key
JOIN table3 ON table1.key = table3.key;
```

### Types of JOINs:
- **INNER JOIN** (default): Returns only matching records from both tables
- **LEFT JOIN**: Returns all records from left table + matching from right
- **RIGHT JOIN**: Returns all records from right table + matching from left  
- **FULL OUTER JOIN**: Returns all records from both tables

### Our Query Breakdown:
```sql
SELECT * FROM motion 
JOIN station ON motion.station_id = station.station_id 
JOIN event ON motion.event_id = event.event_id
```

This creates a **complete picture** by combining:
1. **Motion measurements** (PGA, distance)
2. **Station details** (location, soil properties)  
3. **Event information** (magnitude, epicenter)

### Real-World Applications:
- **Seismic hazard analysis**: Compare ground motion vs distance and magnitude
- **Site response studies**: Analyze how different soil types (Vs30) affect shaking
- **Building code development**: Understand ground motion patterns for design

## Key Takeaways and Next Steps

### What You've Learned:

1. **SQLite Basics**: Database connection, cursor operations, error handling
2. **Database Design**: Primary keys, foreign keys, relational structure
3. **SQL Operations**: CREATE, INSERT, SELECT, JOIN
4. **Data Relationships**: Many-to-many relationships through junction tables
5. **Real-World Application**: Ground motion database for earthquake engineering

### Best Practices Demonstrated:

✅ **Error Handling**: Try/except blocks for robust database operations  
✅ **Safe Table Creation**: Using `IF NOT EXISTS` to prevent conflicts  
✅ **Referential Integrity**: Foreign key constraints maintain data consistency  
✅ **Explicit Column Naming**: Clear, maintainable INSERT statements  
✅ **Meaningful JOINs**: Combining related data for comprehensive analysis  

### Next Steps for Advanced Usage:

1. **Indexes**: Add `CREATE INDEX` for faster queries on large datasets
2. **Transactions**: Use `BEGIN/COMMIT` for multi-statement operations
3. **Parameterized Queries**: Prevent SQL injection with `?` placeholders
4. **Database Migration**: Version control your schema changes
5. **Integration**: Connect with pandas, matplotlib for data analysis

### Performance Optimization:

```sql
-- Add indexes for frequently queried columns
CREATE INDEX idx_motion_event ON motion(event_id);
CREATE INDEX idx_motion_station ON motion(station_id);

-- Use parameterized queries for safety
cursor.execute("SELECT * FROM event WHERE magnitude > ?", (min_mag,))

-- Use transactions for bulk operations
con.execute("BEGIN")
# Multiple INSERT operations
con.execute("COMMIT")
```

### Production Recommendations:

- **Use context managers**: `with sqlite3.connect() as con:`
- **Validate input data**: Check ranges, data types before insertion
- **Regular backups**: SQLite files are easy to backup/restore
- **Monitor file size**: Consider partitioning for very large datasets
- **Test thoroughly**: Validate foreign key relationships and constraints

## Advanced SQL Examples and Data Export/Import

### More Complex Queries with Aggregation and Filtering

Let's explore advanced SQL operations with our ground motion database:

In [9]:
# Example 1: Find average PGA by earthquake event
query = '''
SELECT 
    e.event_name,
    e.magnitude,
    COUNT(m.motion_id) as num_recordings,
    AVG(m.pga) as avg_pga,
    MAX(m.pga) as max_pga,
    MIN(m.rjb) as closest_distance
FROM event e
JOIN motion m ON e.event_id = m.event_id
GROUP BY e.event_id, e.event_name, e.magnitude
ORDER BY e.magnitude DESC
'''

print("📊 Average ground motion by earthquake event:")
print("=" * 50)
exec(cur, query)
results = cur.fetchall()
for row in results:
    print(f"Event: {row[0]}")
    print(f"  Magnitude: {row[1]}")
    print(f"  Recordings: {row[2]}")
    print(f"  Avg PGA: {row[3]:.3f}g")
    print(f"  Max PGA: {row[4]:.3f}g") 
    print(f"  Closest Station: {row[5]:.1f}km")
    print()

📊 Average ground motion by earthquake event:
Event: Hollywood Valley
  Magnitude: 7.2
  Recordings: 2
  Avg PGA: 0.465g
  Max PGA: 0.610g
  Closest Station: 20.0km

Event: San Andreas Test
  Magnitude: 5.8
  Recordings: 2
  Avg PGA: 0.560g
  Max PGA: 0.840g
  Closest Station: 2.0km



In [10]:
# Example 2: Find high ground motion recordings (PGA > 0.5g)
query = '''
SELECT 
    e.event_name,
    s.station_name,
    m.pga,
    m.rjb,
    s.vs30,
    CASE 
        WHEN s.vs30 < 300 THEN 'Soft Soil'
        WHEN s.vs30 < 600 THEN 'Medium Soil' 
        ELSE 'Hard Soil'
    END as soil_type
FROM motion m
JOIN event e ON m.event_id = e.event_id
JOIN station s ON m.station_id = s.station_id
WHERE m.pga > 0.5
ORDER BY m.pga DESC
'''

print("🚨 High ground motion recordings (PGA > 0.5g):")
print("=" * 55)
exec(cur, query)
high_motion = cur.fetchall()
for row in high_motion:
    print(f"Event: {row[0]} | Station: {row[1]}")
    print(f"  PGA: {row[2]:.3f}g | Distance: {row[3]:.1f}km")
    print(f"  Vs30: {row[4]:.0f}m/s ({row[5]})")
    print()

🚨 High ground motion recordings (PGA > 0.5g):
Event: San Andreas Test | Station: Factor Building
  PGA: 0.840g | Distance: 2.0km
  Vs30: 450m/s (Medium Soil)

Event: Hollywood Valley | Station: Factor Building
  PGA: 0.610g | Distance: 20.0km
  Vs30: 450m/s (Medium Soil)



## Integrating SQLite with Pandas

### Why Use Pandas with SQLite?

**Pandas** provides powerful data analysis capabilities that complement SQLite:
- **Data manipulation**: Advanced filtering, grouping, and transformations
- **Visualization**: Easy plotting with matplotlib/seaborn integration
- **Export options**: Multiple file formats (CSV, Excel, JSON, Parquet)
- **Statistical analysis**: Built-in descriptive statistics and correlation analysis

Let's demonstrate the integration:

In [11]:
import pandas as pd
import numpy as np

# Method 1: Read entire tables into pandas DataFrames
print("📋 Loading data into pandas DataFrames:")
print("=" * 40)

# Load each table
events_df = pd.read_sql_query("SELECT * FROM event", con)
stations_df = pd.read_sql_query("SELECT * FROM station", con)
motion_df = pd.read_sql_query("SELECT * FROM motion", con)

print(f"Events table: {len(events_df)} rows")
print(events_df.head())
print(f"\nStations table: {len(stations_df)} rows") 
print(stations_df.head())
print(f"\nMotion table: {len(motion_df)} rows")
print(motion_df.head())

📋 Loading data into pandas DataFrames:
Events table: 2 rows
   event_id        event_name  magnitude  epicentral_latitude  \
0         1  San Andreas Test        5.8              34.0522   
1         2  Hollywood Valley        7.2              34.1027   

   epicentral_longitude  
0             -118.2437  
1             -118.3404  

Stations table: 2 rows
   station_id             station_name  station_latitude  station_longitude  \
0           1          Factor Building         34.006677        -118.442007   
1           2  Santa Monica Courthouse         34.010800        -118.490300   

    vs30  
0  450.0  
1  275.0  

Motion table: 4 rows
   motion_id  event_id  station_id   rjb   pga
0          1         1           1   2.0  0.84
1          2         1           2  14.0  0.28
2          3         2           1  20.0  0.61
3          4         2           2  30.0  0.32


In [12]:
# Method 2: Complex query directly to pandas DataFrame
complex_query = '''
SELECT 
    e.event_name,
    e.magnitude,
    s.station_name,
    s.vs30,
    m.rjb as distance_km,
    m.pga,
    CASE 
        WHEN s.vs30 < 300 THEN 'Soft'
        WHEN s.vs30 < 600 THEN 'Medium' 
        ELSE 'Hard'
    END as soil_class
FROM motion m
JOIN event e ON m.event_id = e.event_id
JOIN station s ON m.station_id = s.station_id
ORDER BY e.magnitude DESC, m.rjb ASC
'''

# Load complex joined data directly into pandas
ground_motion_df = pd.read_sql_query(complex_query, con)

print("🔗 Combined ground motion dataset:")
print("=" * 35)
print(ground_motion_df)
print(f"\nDataset shape: {ground_motion_df.shape}")
print(f"Columns: {list(ground_motion_df.columns)}")

🔗 Combined ground motion dataset:
         event_name  magnitude             station_name   vs30  distance_km  \
0  Hollywood Valley        7.2          Factor Building  450.0         20.0   
1  Hollywood Valley        7.2  Santa Monica Courthouse  275.0         30.0   
2  San Andreas Test        5.8          Factor Building  450.0          2.0   
3  San Andreas Test        5.8  Santa Monica Courthouse  275.0         14.0   

    pga soil_class  
0  0.61     Medium  
1  0.32       Soft  
2  0.84     Medium  
3  0.28       Soft  

Dataset shape: (4, 7)
Columns: ['event_name', 'magnitude', 'station_name', 'vs30', 'distance_km', 'pga', 'soil_class']


In [13]:
# Pandas data analysis examples
print("📈 Data Analysis with Pandas:")
print("=" * 30)

# Basic statistics
print("📊 Descriptive Statistics:")
print(ground_motion_df[['magnitude', 'distance_km', 'pga', 'vs30']].describe())

print("\n🏗️ Ground Motion by Soil Type:")
soil_analysis = ground_motion_df.groupby('soil_class').agg({
    'pga': ['mean', 'std', 'max'],
    'vs30': 'mean',
    'distance_km': 'mean'
}).round(3)
print(soil_analysis)

print("\n🎯 Correlation Analysis:")
correlations = ground_motion_df[['magnitude', 'distance_km', 'pga', 'vs30']].corr()
print(correlations.round(3))

📈 Data Analysis with Pandas:
📊 Descriptive Statistics:
       magnitude  distance_km       pga        vs30
count    4.00000       4.0000  4.000000    4.000000
mean     6.50000      16.5000  0.512500  362.500000
std      0.80829      11.7047  0.263233  101.036297
min      5.80000       2.0000  0.280000  275.000000
25%      5.80000      11.0000  0.310000  275.000000
50%      6.50000      17.0000  0.465000  362.500000
75%      7.20000      22.5000  0.667500  450.000000
max      7.20000      30.0000  0.840000  450.000000

🏗️ Ground Motion by Soil Type:
              pga                vs30 distance_km
             mean    std   max   mean        mean
soil_class                                       
Medium      0.725  0.163  0.84  450.0        11.0
Soft        0.300  0.028  0.32  275.0        22.0

🎯 Correlation Analysis:
             magnitude  distance_km    pga   vs30
magnitude        1.000        0.839 -0.208  0.000
distance_km      0.839        1.000 -0.695 -0.543
pga             -0.2

## Data Export: From SQLite to Various Formats

### CSV Export (Most Common)

In [14]:
# Export to CSV files
print("💾 Exporting data to CSV format:")
print("=" * 35)

# Export individual tables
events_df.to_csv('earthquake_events.csv', index=False)
stations_df.to_csv('seismic_stations.csv', index=False)
motion_df.to_csv('ground_motion_data.csv', index=False)

# Export combined dataset
ground_motion_df.to_csv('combined_ground_motion.csv', index=False)

print("✅ CSV files created:")
print("  - earthquake_events.csv")
print("  - seismic_stations.csv") 
print("  - ground_motion_data.csv")
print("  - combined_ground_motion.csv")

# Show CSV content preview
print(f"\n📄 Preview of combined_ground_motion.csv:")
with open('combined_ground_motion.csv', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        print(f"  {line.strip()}")
        if i >= 4:  # Show first 5 lines
            break

💾 Exporting data to CSV format:
✅ CSV files created:
  - earthquake_events.csv
  - seismic_stations.csv
  - ground_motion_data.csv
  - combined_ground_motion.csv

📄 Preview of combined_ground_motion.csv:
  event_name,magnitude,station_name,vs30,distance_km,pga,soil_class
  Hollywood Valley,7.2,Factor Building,450.0,20.0,0.61,Medium
  Hollywood Valley,7.2,Santa Monica Courthouse,275.0,30.0,0.32,Soft
  San Andreas Test,5.8,Factor Building,450.0,2.0,0.84,Medium
  San Andreas Test,5.8,Santa Monica Courthouse,275.0,14.0,0.28,Soft


### Excel Export (Multiple Sheets)

In [15]:
# Export to Excel with multiple sheets
print("📊 Exporting data to Excel format:")
print("=" * 35)

try:
    # Create Excel file with multiple sheets
    with pd.ExcelWriter('ground_motion_database.xlsx', engine='openpyxl') as writer:
        # Write each table to a separate sheet
        events_df.to_excel(writer, sheet_name='Events', index=False)
        stations_df.to_excel(writer, sheet_name='Stations', index=False)
        motion_df.to_excel(writer, sheet_name='GroundMotion', index=False)
        ground_motion_df.to_excel(writer, sheet_name='Combined', index=False)
        
        # Create a summary sheet
        summary_data = {
            'Table': ['Events', 'Stations', 'Ground Motion', 'Combined Dataset'],
            'Rows': [len(events_df), len(stations_df), len(motion_df), len(ground_motion_df)],
            'Columns': [len(events_df.columns), len(stations_df.columns), 
                       len(motion_df.columns), len(ground_motion_df.columns)]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print("✅ Excel file created: ground_motion_database.xlsx")
    print("   📋 Sheets included:")
    print("     - Events (earthquake data)")
    print("     - Stations (recording stations)")
    print("     - GroundMotion (raw measurements)")
    print("     - Combined (joined dataset)")
    print("     - Summary (data overview)")
    
except ImportError:
    print("❌ Excel export requires openpyxl library")
    print("   Install with: pip install openpyxl")
except Exception as e:
    print(f"❌ Excel export failed: {e}")

📊 Exporting data to Excel format:
✅ Excel file created: ground_motion_database.xlsx
   📋 Sheets included:
     - Events (earthquake data)
     - Stations (recording stations)
     - GroundMotion (raw measurements)
     - Combined (joined dataset)
     - Summary (data overview)


## Data Import: From Files Back to SQLite

### Reading CSV Files and Importing to Database

In [16]:
# Demonstrate importing CSV data back to SQLite
print("📥 Importing CSV data back to SQLite:")
print("=" * 38)

# Create a new database for the import demo
import_con = sqlite3.connect('imported_gmdatabase.db')
import_cur = import_con.cursor()

# Read CSV file and import to new database
try:
    # Read CSV file
    imported_df = pd.read_csv('combined_ground_motion.csv')
    print(f"✅ Read CSV file: {len(imported_df)} rows")
    
    # Write DataFrame to SQLite table
    imported_df.to_sql('ground_motion_analysis', import_con, 
                      if_exists='replace', index=False)
    
    # Verify the import
    verify_query = "SELECT COUNT(*) FROM ground_motion_analysis"
    import_cur.execute(verify_query)
    row_count = import_cur.fetchone()[0]
    
    print(f"✅ Data imported to new database: {row_count} rows")
    
    # Show table schema
    schema_query = "PRAGMA table_info(ground_motion_analysis)"
    import_cur.execute(schema_query)
    schema_info = import_cur.fetchall()
    
    print("\n📋 Imported table schema:")
    for col_info in schema_info:
        print(f"  {col_info[1]} ({col_info[2]})")
    
    # Test query on imported data
    test_query = "SELECT event_name, station_name, pga FROM ground_motion_analysis LIMIT 3"
    import_cur.execute(test_query)
    sample_data = import_cur.fetchall()
    
    print("\n📄 Sample imported data:")
    for row in sample_data:
        print(f"  {row[0]} | {row[1]} | PGA: {row[2]}")
        
except FileNotFoundError:
    print("❌ CSV file not found. Run the export section first.")
except Exception as e:
    print(f"❌ Import failed: {e}")
finally:
    import_con.close()

📥 Importing CSV data back to SQLite:
✅ Read CSV file: 4 rows
✅ Data imported to new database: 4 rows

📋 Imported table schema:
  event_name (TEXT)
  magnitude (REAL)
  station_name (TEXT)
  vs30 (REAL)
  distance_km (REAL)
  pga (REAL)
  soil_class (TEXT)

📄 Sample imported data:
  Hollywood Valley | Factor Building | PGA: 0.61
  Hollywood Valley | Santa Monica Courthouse | PGA: 0.32
  San Andreas Test | Factor Building | PGA: 0.84


### Reading Excel Files and Advanced Import Options

In [17]:
# Reading Excel files and importing to SQLite
print("📊 Reading Excel files and importing to SQLite:")
print("=" * 47)

try:
    # Read specific sheets from Excel file
    excel_file = 'ground_motion_database.xlsx'
    
    # Method 1: Read all sheets
    all_sheets = pd.read_excel(excel_file, sheet_name=None)
    print(f"✅ Excel file contains {len(all_sheets)} sheets:")
    for sheet_name in all_sheets.keys():
        rows = len(all_sheets[sheet_name])
        print(f"  📋 {sheet_name}: {rows} rows")
    
    # Method 2: Read specific sheet
    events_from_excel = pd.read_excel(excel_file, sheet_name='Events')
    print(f"\n✅ Read 'Events' sheet: {len(events_from_excel)} rows")
    print("First few rows:")
    print(events_from_excel.head(2))
    
    # Method 3: Import Excel data to new SQLite database
    excel_con = sqlite3.connect('excel_imported_data.db')
    
    # Import each sheet as a separate table
    for sheet_name, data in all_sheets.items():
        if sheet_name != 'Summary':  # Skip summary sheet
            table_name = f"excel_{sheet_name.lower()}"
            data.to_sql(table_name, excel_con, if_exists='replace', index=False)
            print(f"✅ Imported {sheet_name} → {table_name} table")
    
    excel_con.close()
    print("\n🎉 Excel import completed successfully!")
    
except FileNotFoundError:
    print("❌ Excel file not found. Run the Excel export section first.")
except ImportError:
    print("❌ Excel reading requires openpyxl library")
    print("   Install with: pip install openpyxl")
except Exception as e:
    print(f"❌ Excel import failed: {e}")

📊 Reading Excel files and importing to SQLite:
✅ Excel file contains 5 sheets:
  📋 Events: 2 rows
  📋 Stations: 2 rows
  📋 GroundMotion: 4 rows
  📋 Combined: 4 rows
  📋 Summary: 4 rows

✅ Read 'Events' sheet: 2 rows
First few rows:
   event_id        event_name  magnitude  epicentral_latitude  \
0         1  San Andreas Test        5.8              34.0522   
1         2  Hollywood Valley        7.2              34.1027   

   epicentral_longitude  
0             -118.2437  
1             -118.3404  
✅ Imported Events → excel_events table
✅ Imported Stations → excel_stations table
✅ Imported GroundMotion → excel_groundmotion table
✅ Imported Combined → excel_combined table

🎉 Excel import completed successfully!


## Production-Ready Code Examples

### Critical Best Practices for Real Applications

When moving from tutorial code to production systems, several important practices ensure your database operations are **secure**, **reliable**, and **maintainable**:

#### 🔒 **1. Security Best Practices**
- **Parameterized Queries**: Use `?` placeholders instead of string formatting to prevent SQL injection attacks
- **Input Validation**: Always validate data types, ranges, and required fields before database operations
- **Access Control**: Enable foreign key constraints and use appropriate user permissions

#### 🛡️ **2. Error Handling & Reliability**
- **Transaction Management**: Use `BEGIN/COMMIT/ROLLBACK` for multi-step operations
- **Connection Management**: Properly close connections and handle connection failures
- **Exception Handling**: Catch specific database errors and provide meaningful error messages
- **Data Integrity**: Validate business rules and data constraints before insertion

#### ⚡ **3. Performance Optimization**
- **Connection Pooling**: Reuse database connections for multiple operations
- **Prepared Statements**: Compile SQL once, execute multiple times efficiently
- **Indexes**: Create appropriate indexes for frequently queried columns
- **Bulk Operations**: Use batch inserts for large datasets

#### 🏗️ **4. Code Architecture**
- **Context Managers**: Use `with` statements or custom context managers for automatic cleanup
- **Function Separation**: Separate validation, database operations, and business logic
- **Configuration**: Store database paths and settings in configuration files
- **Logging**: Implement proper logging for debugging and monitoring

#### 📊 **5. Data Quality & Validation**
- **Schema Validation**: Ensure data matches expected types and constraints
- **Range Checking**: Validate that numerical values fall within realistic ranges
- **Required Fields**: Check for missing or null values in critical fields
- **Referential Integrity**: Ensure foreign key relationships are maintained

### What the Following Code Demonstrates:

The next cell shows a **production-ready implementation** that incorporates all these best practices:

1. **Safe Database Context Manager** - Handles connections, transactions, and cleanup automatically
2. **Input Validation Function** - Validates earthquake data before insertion
3. **Parameterized Queries** - Prevents SQL injection attacks
4. **Error Handling** - Graceful handling of database errors with rollback capability
5. **Data Range Validation** - Ensures realistic values for earthquake parameters
6. **Foreign Key Enforcement** - Enables database constraints for data integrity

This approach transforms simple tutorial code into **best-practice** database operations suitable for real scientific applications.

In [20]:
# Production-ready database operations
print("🔧 Production-Ready Database Operations:")
print("=" * 41)

def safe_database_operation(db_path, operation_func):
    """Context manager for safe database operations"""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        conn.execute("PRAGMA foreign_keys = ON")  # Enable foreign key constraints
        result = operation_func(conn)
        conn.commit()
        return result
    except sqlite3.Error as e:
        if conn:
            conn.rollback()
        print(f"❌ Database error: {e}")
        return None
    finally:
        if conn:
            conn.close()

def insert_event_safely(conn, event_data):
    """Safely insert earthquake event with validation"""
    # Validate input data
    required_fields = ['event_name', 'magnitude', 'epicentral_latitude', 'epicentral_longitude']
    for field in required_fields:
        if field not in event_data:
            raise ValueError(f"Missing required field: {field}")
    
    # Validate ranges
    if not (-90 <= event_data['epicentral_latitude'] <= 90):
        raise ValueError("Invalid latitude: must be between -90 and 90")
    if not (-180 <= event_data['epicentral_longitude'] <= 180):
        raise ValueError("Invalid longitude: must be between -180 and 180")
    if not (0 <= event_data['magnitude'] <= 10):
        raise ValueError("Invalid magnitude: must be between 0 and 10")
    
    # Use parameterized query (prevents SQL injection)
    query = '''
    INSERT INTO event (event_name, magnitude, epicentral_latitude, epicentral_longitude)
    VALUES (?, ?, ?, ?)
    '''
    
    cursor = conn.cursor()
    cursor.execute(query, (
        event_data['event_name'],
        event_data['magnitude'],
        event_data['epicentral_latitude'],
        event_data['epicentral_longitude']
    ))
    
    return cursor.lastrowid

# Add new event to the database
new_event = {
    'event_name': 'San Andreas Test',
    'magnitude': 5.8,
    'epicentral_latitude': 34.0522,
    'epicentral_longitude': -118.2437
}

result = safe_database_operation('gmdatabase.db', 
                                lambda conn: insert_event_safely(conn, new_event))

if result:
    print(f"✅ Safely inserted new event with ID: {result}")
else:
    print("❌ Failed to insert event")

🔧 Production-Ready Database Operations:
✅ Safely inserted new event with ID: 2


In [21]:
# Clean up: Close the original connection
con.close()

print("\n🧹 Database connections closed")
print("✅ Tutorial completed successfully!")


🧹 Database connections closed
✅ Tutorial completed successfully!
